### Mercari Price 
The files consist of a list of product listings. These files are tab-delimited.

Fields:
- train_id or test_id - the id of the listing

- name - the title of the listing. Note that we have cleaned the data to remove text that look like prices (e.g. $20) to avoid  leakage. These removed prices are represented as [rm]

- item_condition_id - the condition of the items provided by the seller

- category_name - category of the listing

- brand_name

- price - the price that the item was sold for. This is the target variable that you will predict. The unit is USD. This column doesn't exist in test.tsv since that is what you will predict.

- shipping - 1 if shipping fee is paid by seller and 0 by buyer

- item_description - the full description of the item. Note that we have cleaned the data to remove text that look like prices (e.g. $20) to avoid leakage. These removed prices are represented as [rm]

In [118]:
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
import nltk
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


from tqdm._tqdm_notebook import tqdm_notebook

import os
import itertools

import matplotlib.pyplot as plt
import itertools
from collections import Counter
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
tqdm_notebook.pandas()

/home/rpo/workspace/advancedml2021/project-mercari-price/venv/lib/python3.8/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### Dataset download

In [3]:
# check if in colab
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    print("Running in colab")
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)
    colab_root = '/content/drive'
    root_dir = "/content/gdrive/My Drive/"
    base_dir = root_dir + 'project-mercari-price/'
    if not os.path.isdir(base_dir):
        os.mkdir(base_dir)
else:
    root_dir= os.getcwd()
    base_dir = root_dir
    
os.chdir(base_dir)

In [4]:
dataset_downloaded_path = os.path.join(base_dir, "dataset_downloaded.ignore")
dataset_downloaded = os.path.isfile(dataset_downloaded_path)
dataset_downloaded

True

In [5]:
if not dataset_downloaded:
  # install kaggle to download dataset
  ! pip install kaggle python-dotenv

In [6]:
# set to True if you want to save kaggle credentials into a .env file
persist_credentials = False

if not dataset_downloaded:
  # create .env file containing KAGGLE_USER and KAGGLE_KEY
    kaggle_env = os.path.join(base_dir, '.env')
    if not os.path.isfile(kaggle_env):
        with open(kaggle_env, 'w') as envfile:
            kaggle_user = input("Insert kaggle username")
            kaggle_key = input("Insert kaggle key; generate one from kaggle account")
        if persist_credentials:
            envfile.write(f"""
            KAGGLE_USERNAME={kaggle_user}
            KAGGLE_KEY={kaggle_key}
            """)

        # set env vars
        os.environ["KAGGLE_USERNAME"] = kaggle_user
        os.environ["KAGGLE_KEY"] = kaggle_key

        del kaggle_user
        del kaggle_key

In [7]:
if not dataset_downloaded:
  # loading env vars if .env file exists
    if os.path.isfile(kaggle_env):
        from dotenv import load_dotenv
        load_dotenv(dotenv_path=kaggle_env)
    print(os.environ.get("KAGGLE_USERNAME"))

In [8]:
if not dataset_downloaded:
    # download and extract dataset
    ! kaggle competitions download -c mercari-price-suggestion-challenge

    # create file so that we know we already downloaded
    with open(dataset_downloaded_path, 'w') as dd_file:
        dataset_downloaded = True
        dd_file.write("")

    print('cwd: ', os.getcwd())
    
    os.listdir()

In [9]:
if not dataset_downloaded:
    ! 7z x train.tsv.7z
    ! 7z x test.tsv.7z

In [10]:
os.listdir()

['sample_submission.csv.7z',
 'MercariPrice.ipynb',
 'train.tsv',
 'requirements.txt',
 'venv',
 'test.tsv.7z',
 'mercari-price-suggestion-challenge.zip',
 'Readme.md',
 '.gitignore',
 'dataset_downloaded.ignore',
 'train.tsv.7z',
 '.ipynb_checkpoints',
 'test.tsv',
 'test_stg2.tsv.zip',
 'sample_submission_stg2.csv.zip',
 '.git']

## Load dataset

In [46]:
dtypes={
    'name': 'string',
    'item_condition_id': 'byte',
    'category_name': 'string',
    'brand_name': 'string',
    'price': 'float',
    'shipping': 'boolean',
    'item_description': 'string'
}
data = pd.read_csv("train.tsv", sep='\t', dtype=dtypes)
data = data.drop(columns=["train_id"])
data

,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,<NA>,10.0,True,No description yet
1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,False,This keyboard is in great condition and works ...
2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,True,Adorable top with a hint of lace and a key hol...
3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,<NA>,35.0,True,New with tags. Leather horses. Retail for [rm]...
4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,<NA>,44.0,False,Complete with certificate of authenticity
...,...,...,...,...,...,...,...
1482530,Free People Inspired Dress,2,Women/Dresses/Mid-Calf,Free People,20.0,True,"Lace, says size small but fits medium perfectl..."
1482531,Little mermaid handmade dress,2,Kids/Girls 2T-5T/Dresses,Disney,14.0,False,Little mermaid handmade dress never worn size 2t
1482532,21 day fix containers and eating plan,2,Sports & Outdoors/Exercise/Fitness accessories,<NA>,12.0,False,"Used once or twice, still in great shape."
1482533,World markets lanterns,3,Home/Home Décor/Home Décor Accents,<NA>,45.0,True,There is 2 of each one that you see! So 2 red ...


In [47]:
data.dtypes

name                  string
item_condition_id       int8
category_name         string
brand_name            string
price                float64
shipping             boolean
item_description      string
dtype: object

In [48]:
data.shape

(1482535, 7)

In [49]:
for column in data.columns:
    print("number of null value in {} : {}".format(column,data[column].isnull().sum()))

number of null value in name : 0
number of null value in item_condition_id : 0
number of null value in category_name : 6327
number of null value in brand_name : 632682
number of null value in price : 0
number of null value in shipping : 0
number of null value in item_description : 4


In [50]:
data = data[data["item_description"].notna()]
data["brand_name"] = data["brand_name"].fillna(value="NA")
data["category_name"] = data["category_name"].fillna(value="NA")
# see warnings -> inplace?
data.shape

<ipython-input-50-73e1feea4d04>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["brand_name"] = data["brand_name"].fillna(value="NA")
<ipython-input-50-73e1feea4d04>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["category_name"] = data["category_name"].fillna(value="NA")


(1482531, 7)

In [51]:
def flat_list(l):
    return  [item for sublist in l for item in sublist]


def plot_common_tokens(tokens, title, n=20):
    sentences = (list(itertools.chain(tokens)))
    flat_sentences = flat_list(sentences)
    counts = Counter(flat_sentences)
    #print(counts.most_common(30))
    common_words = [word[0] for word in counts.most_common(n)]
    common_counts = [word[1] for word in counts.most_common(n)]
    fig = plt.figure(figsize=(18,6))
    sns.barplot(x=common_words, y=common_counts)
    plt.title(title)
    plt.show()


In [52]:
#plot_common_tokens(data["item_description_tokens"], "Most Common Tokens from Descriptions")

In [53]:
X = data[['name', 'item_condition_id', 'category_name', 'brand_name',
       'shipping','item_description']]
y = data['price']

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=1000)

In [24]:
def getTokens(data, tokenizer=None, dataFit=None, num_words=5000):
    if dataFit is None:
        dataFit = data
        
    if tokenizer is None:
        tokenizer = Tokenizer(num_words=num_words)
        tokenizer.fit_on_texts(dataFit)
    
    tokens = tokenizer.texts_to_sequences(data)
    return tokens, tokenizer

In [27]:
tokenizers = {}

In [127]:
X_train["item_description_t"], tokenizers["item_description"] = getTokens(X_train["item_description"])
X_train["name_t"], tokenizers["name"] = getTokens(X_train["name"])
#X_train["brand_name_t"], tokenizers["brand_name"] = getTokens(X_train["brand_name"])
#X_train["category_name_t"], tokenizers["category_name"] = getTokens(X_train["category_name"])

In [128]:
tokenizers

{'item_description': <keras_preprocessing.text.Tokenizer at 0x7f3ee941d820>,
 'name': <keras_preprocessing.text.Tokenizer at 0x7f3f9869a8e0>,
 'brand_name': <keras_preprocessing.text.Tokenizer at 0x7f3f986a9190>,
 'category_name': <keras_preprocessing.text.Tokenizer at 0x7f3f0fd0adc0>}

In [129]:
X_train["item_description_t"]

995196     [6, 5, 144, 279, 173, 979, 84, 135, 173, 657, ...
28823      [685, 1020, 340, 78, 306, 50, 6, 281, 90, 2526...
661855     [27, 17, 136, 170, 194, 1102, 613, 188, 759, 6...
733730     [5, 444, 1202, 99, 16, 86, 177, 16, 12, 165, 1...
807837     [27, 106, 497, 11, 435, 117, 235, 4571, 682, 2...
                                 ...                        
371719                              [9, 83, 232, 35, 32, 26]
1311257    [3867, 3867, 2095, 23, 3, 146, 3928, 2, 1827, ...
404942                     [6, 49, 540, 167, 41, 1340, 1878]
242521     [200, 128, 69, 2977, 124, 234, 362, 96, 133, 9...
371320     [2, 1450, 761, 2116, 5, 10, 3, 153, 494, 494, ...
Name: item_description_t, Length: 889518, dtype: object

LabelEncoder()

In [102]:
cat_le = preprocessing.LabelEncoder()
cat_le.fit(X_train["category_name"])

X_train["category_name_l"] = cat_le.transform(X_train["category_name"])

In [105]:
brand_le = preprocessing.LabelEncoder()
brand_le.fit(X_train["brand_name"])

X_train["brand_name_l"] = brand_le.transform(X_train["brand_name"])

In [57]:
len(tokenizers["item_description"].word_index)

158136

In [106]:
# unknown values !!! now -> crash

In [36]:
X_train.loc[995196, ["name", "name_t"]]

name      PINK❤️warm and cozy mist/lotion set
name_t                [9, 1995, 431, 288, 17]
Name: 995196, dtype: object

In [111]:
X_train.head()

,name,item_condition_id,category_name,brand_name,shipping,item_description,item_description_t,name_t,category_name_l,brand_name_l
995196,PINK❤️warm and cozy mist/lotion set,1,Beauty/Fragrance/Women,Victoria's Secret,False,New in original packaging -Body mist 250ml/8.4...,"[6, 5, 144, 279, 173, 979, 84, 135, 173, 657, ...","[9, 1995, 431, 288, 17]",11,3885
28823,18 min vest,3,Kids/Girls 0-24 Mos/One-Pieces,NA,True,Faux fur zipper up best like new Purchased at ...,"[685, 1020, 340, 78, 306, 50, 6, 281, 90, 2526...","[234, 276]",596,2585
661855,Artsy stylish leather purse & pouch,3,Handmade/Bags and Purses/Purse,NA,False,Used condition cowhide high quality leather!! ...,"[27, 17, 136, 170, 194, 1102, 613, 188, 759, 6...","[3172, 118, 83, 922]",157,2585
733730,Decor pillows,2,Home/Home Décor/Decorative Pillows,NA,False,In mint conditions Smoke free home Pet free No...,"[5, 444, 1202, 99, 16, 86, 177, 16, 12, 165, 1...","[599, 2390]",453,2585
807837,IF NOT SOLD DONATING 2/6,4,Men/Shoes/Outdoor,Oakley,False,Used still lots of life...see pictures...Oakle...,"[27, 106, 497, 11, 435, 117, 235, 4571, 682, 2...","[4155, 630, 1029, 11, 20]",733,2714


In [116]:
desc_vocab_size = len(tokenizers['item_description'].word_index) + 1
print(desc_vocab_size)

name_vocab_size = len(tokenizers['name'].word_index) + 1
print(name_vocab_size)

158137
88696


In [119]:
X_train["item_description_t"]

995196     [6, 5, 144, 279, 173, 979, 84, 135, 173, 657, ...
28823      [685, 1020, 340, 78, 306, 50, 6, 281, 90, 2526...
661855     [27, 17, 136, 170, 194, 1102, 613, 188, 759, 6...
733730     [5, 444, 1202, 99, 16, 86, 177, 16, 12, 165, 1...
807837     [27, 106, 497, 11, 435, 117, 235, 4571, 682, 2...
                                 ...                        
371719                              [9, 83, 232, 35, 32, 26]
1311257    [3867, 3867, 2095, 23, 3, 146, 3928, 2, 1827, ...
404942                     [6, 49, 540, 167, 41, 1340, 1878]
242521     [200, 128, 69, 2977, 124, 234, 362, 96, 133, 9...
371320     [2, 1450, 761, 2116, 5, 10, 3, 153, 494, 494, ...
Name: item_description_t, Length: 889518, dtype: object

In [140]:
test = X_train["item_description_t"]
padded = pad_sequences(test, padding='post', maxlen=20)

In [246]:

maxlen = 100

inputDesc_train= pad_sequences(X_train["item_description_t"],
                                                  padding='post', maxlen=maxlen)

In [247]:
inputName_train = pad_sequences(X_train["name_t"], padding='post', maxlen=20)

In [164]:
import tensorflow.keras.backend as K
msle = tf.keras.losses.MeanSquaredLogarithmicError()

def root_mean_squared_logarithmic_error(y_true, y_pred):
    return K.sqrt(msle(y_true, y_pred))


In [198]:
y_train

995196      24.0
28823       13.0
661855     126.0
733730      12.0
807837      13.0
           ...  
371719      24.0
1311257     10.0
404942      10.0
242521      23.0
371320       8.0
Name: price, Length: 889518, dtype: float64

In [175]:
from tensorflow.keras.layers import Dense, Input, Embedding, Concatenate, Flatten
from tensorflow.keras.models import Model

In [173]:
desc_vocab_size

158137

In [255]:
X_train[["item_condition_id", "category_name_l", "brand_name_l", "shipping"]]

,item_condition_id,category_name_l,brand_name_l,shipping
995196,1,11,3885,False
28823,3,596,2585,True
661855,3,157,2585,False
733730,2,453,2585,False
807837,4,733,2714,False
...,...,...,...,...
371719,2,699,2180,False
1311257,1,652,1705,False
404942,1,28,2585,False
242521,2,1217,3885,True


In [184]:
def getModel():
    inputA = Input(shape=(4,))
    Ad = Dense(4, activation='relu')(inputA)
    
    inputName = Input(shape=(20,))
    Ne = Embedding(input_dim=name_vocab_size, output_dim=16, input_length=20)(inputName)
    Nf = Flatten()(Ne)
    Nd = Dense(8, activation='relu')(Nf)
    
    inputDesc = Input(shape=(100,))
    De = Embedding(input_dim=desc_vocab_size, output_dim=32, input_length=100)(inputDesc)
    Df = Flatten()(De)
    Dd = Dense(16, activation='relu')(Df)
    
    concat = Concatenate()([Ad, Nd, Dd])
    
    x = Dense(32, activation='relu')(concat)
    x = Dense(1, activation='linear')(x)
    
    model = Model(inputs=[inputA, inputName, inputDesc], outputs=x)
    
    return model
    

In [186]:
model = getModel()
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 20, 16)       1419136     input_16[0][0]                   
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 100, 32)      5060384     input_17[0][0]                   
_______________________________________________________________________________________

In [194]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError(), 'mean_squared_logarithmic_error', root_mean_squared_logarithmic_error])

In [267]:
inputA_train = X_train[["item_condition_id", "category_name_l", "brand_name_l", "shipping"]].values.astype('int32')

In [266]:
inputA_train.astype('int32')

array([[   1,   11, 3885,    0],
       [   3,  596, 2585,    1],
       [   3,  157, 2585,    0],
       ...,
       [   1,   28, 2585,    0],
       [   2, 1217, 3885,    1],
       [   1,   33, 2585,    0]], dtype=int32)

In [ ]:
history = model.fit(x=[inputA_train, inputName_train, inputDesc_train], y=y_train,
                    epochs=5,
                    verbose=True,
                    #validation_data=(X_validation, y_validation),
                    batch_size=256)

Epoch 1/5


In [ ]:
#history1= history

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)